In [1]:
!pip install -U torch==1.8.0 torchtext==0.9.0 --user

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy


# Import Required Libraries & Data Loading

In [4]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [5]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [8]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
####https://spacy.io/usage   :)
spacy.load("en_core_web_sm")

In [9]:
# general Settings
# https://colab.research.google.com/drive/1eaoqM7V_TyBSZCbs32g77r5HXvMGMVYh?usp=sharing
# https://colab.research.google.com/drive/1z0kizW_pCI41xl_OGLqPOyZDEbOvJ_Lv?usp=sharing
# using these 2 ^^ colab as reference

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005 ### ADD YOUR LEARNING RATE HERE ###
BATCH_SIZE = 1 ### ADD YOUR BATCH SIZE HERE ###
NUM_EPOCHS = 100### ADD YOUR NUMBER OF EPOCHS HERE ###
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 400 ### ADD YOUR EMBEDDING DIMENSION HERE ###
HIDDEN_DIM = 256### ADD YOUR HIDDEN DIMENSION HERE ###
NUM_CLASSES = 2

# Text & label Preparation

In [10]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [11]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [12]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = "IMDB Dataset.csv",format = "csv" ,
                    skip_header = True,
                    fields = fields
)

# Data Split

In [14]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [15]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [16]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': 'positive'}


In [17]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [18]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
#https://docs.python.org/3/library/collections.html
#https://pytorch.org/text/stable/vocab.html
#https://www.techwithtim.net/tutorials/python-programming/intermediate-python-tutorials/collections-counter/
#https://pythontic.com/containers/counter/most_common
#https://torchtext.readthedocs.io/en/latest/vocab.html
#^ I looked at a few resources to figure out most_common. I commented out the old code to show my work and process.
#the last resource on torchtext.readthedocs i think is the correct path :)
"""
import collections
from collections import Counter
print("DONE IMPORT")
counter = Counter(TEXT.vocab)
print(counter)
most_common_words =  counter.most_common(3)
print(most_common_words)
"""

most_common_words = TEXT.vocab.freqs.most_common()
print(most_common_words)

[('the', 390972), (',', 369444), ('.', 318519), ('a', 210502), ('and', 210008), ('of', 194659), ('to', 180163), ('is', 145895), ('in', 118266), ('I', 105681), ('it', 103588), ('that', 93995), ('"', 85535), ("'s", 83149), ('this', 81775), ('-', 71249), ('/><br', 68787), ('was', 67372), ('as', 57734), ('movie', 57572), ('with', 57543), ('for', 56557), ('film', 52382), ('The', 51251), ('but', 46575), ('(', 44813), ('on', 44528), ("n't", 44474), (')', 44145), ('you', 42196), ('are', 40223), ('not', 38706), ('have', 38273), ('his', 37471), ('be', 35957), ('he', 32974), ('one', 32838), ('!', 30736), ('at', 29683), ('by', 29553), ('all', 28574), ('an', 28189), ('who', 27643), ('from', 26375), ('like', 26348), ('they', 26269), ('so', 24036), ('or', 23054), ('about', 22864), ('out', 22811), ('has', 22740), ('her', 22621), ("'", 22611), ('just', 22328), ('It', 22291), ('do', 21258), ('?', 20344), ('some', 19437), ('good', 19065), ('more', 18587), ('very', 18035), ('would', 17866), ('up', 17476),

In [19]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [20]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE , ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = None , ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )
#https://www.w3schools.com/python/python_tuples.asp
#i fotgot the basics of a tuple lol ^^
#https://torchtext.readthedocs.io/en/latest/data.html

In [21]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([58, 1])
Target vector size: torch.Size([1])

Valid:
Text matrix size: torch.Size([5, 1])
Target vector size: torch.Size([1])

Test:
Text matrix size: torch.Size([8, 1])
Target vector size: torch.Size([1])


In [22]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_loader, valid_loader, test_loader),
    batch_size = BATCH_SIZE,
    sort_within_batch = None ,
    sort_key = lambda x : len(x.TEXT_COLUMN_NAME),
    device = DEVICE
    )
#https://www.w3schools.com/python/python_tuples.asp
#i fotgot the basics of a tuple lol ^^
#https://torchtext.readthedocs.io/en/latest/data.html


# Model Building

In [23]:
#https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/977c14818c75427641ccb85ad21ed6dc/char_rnn_classification_tutorial.ipynb#scrollTo=Exn_X8uMxjrJ
#I based this section on this colab
#in the colab, you talked about the RNN being from the PyTorch for Torch users tutorial
# So i also used it as a reference 
#https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#sphx-glr-beginner-former-torchies-nnft-tutorial-py

import torch.nn as nn
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        self.hidden_dim = hidden_dim

        self.i2h = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.i2o = nn.Linear(input_dim + hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        
        # embedded dim: [sentence length, batch size, embedding dim]
        
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        # hidden dim: [batch size, hidden dim]
        input = torch.cat((data, last_hidden), 1)
        hidden = self.i2h(input)
        output = self.h2o(hidden)
        #return hidden, output

        
        ### END YOUR CODE ###
        return output

In [24]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=VOCABULARY_SIZE, ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim=EMBEDDING_DIM, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=HIDDEN_DIM, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=NUM_CLASSES  ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)### ADD YOUR OPTIMIZER HERE ###


# Define Accuracy

In [25]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [26]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        h = tuple([each.data for each in h])
        model.zero_grad()      
        output,h = model(text,h)
        optimizer.step()
        
        ### UPDATE MODEL PARAMETERS
        
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

TypeError: 'BucketIterator' object is not subscriptable

# Model Testing

In [ ]:
import spacy

nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")